# Описание проекта

Интернет-магазин запускает новый сервис в котором пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.
Проект нацелен на создание инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. Успешной моделью будет считаться та, которая достигнет метрики F1 не менее 0.75 на тестовой выборке.

---

Описание данных:
- `Unnamed: 0` — Столбец индекса, не несет полезной информации
- `text` — Текст комментария
- `toxic` — Целевая переменная, 1 — токсичный комментарий, 0 — нет.

In [12]:
#imports
import pandas as pd
import numpy as np

In [13]:
# Игнорирование предупреждений:
import warnings
warnings.filterwarnings("ignore")

# Настройки воспроизводимости:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
# torch.manual_seed(RANDOM_SEED)

## Загрузка данных

Загрузим данные и выведем информацию о них.

In [14]:
# Загружаем данные
df = pd.read_csv('data/toxic_comments.csv')

# Выводим датафрейм
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [15]:
# Выводим информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [16]:
# Удаляем столбец
df = df.drop(columns=['Unnamed: 0'], axis=1)

# Приводим к типу int8 для экономии памяти
df = df['toxic'].astype('int8')

# Проверяем, корректно ли удалился столбец
df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 159292 entries, 0 to 159291
Series name: toxic
Non-Null Count   Dtype
--------------   -----
159292 non-null  int8 
dtypes: int8(1)
memory usage: 155.7 KB
